# Prediction of drug target interaction for drug discovery

### Introduction

This notebook aims to provide a baseline for the challenge, by proposing a simple pipeline based on ECFP fingerprints combined with a Random Forest algorithm. The objective is to predict the pIC50 of various drugs on a protein. 

Some tips :
- ⭐ RDKit is a useful package for chemoinformatics and is used in this notebook, you can check the "[Getting started](https://www.rdkit.org/docs/GettingStartedInPython.html)" page on RDKit's website to learn more. Many more packages exist, especially for using deep learning algorithms dedicated to cheminformatics, do not hesitate do do some research ! <br>
- ⭐ Here we use a specific type of fingerprint (ECFP) but many more exist, feel free to try them and even combine them ! <br>
- ⭐ If you have any questions, you can contact me directly by email : pablo.mas@sanofi.com 


### Imports

You can uncomment and run the cell below to install the latest version of the RDKit package.

In [9]:
# !pip install rdkit-pypi

In [10]:
# Linear algebra and data handling
import numpy as np
import pandas as pd

# RDKit
import rdkit 
from rdkit import Chem
from rdkit.Chem import AllChem

# Machine learning
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_validate

### Data loading and preprocessing

First, we need to load the dataset. Do not hesitate to do more exploratory data analysis on your own !

In [11]:
# Loading the train and test csv files
train = pd.read_csv('../data/X_train.csv')
test = pd.read_csv('../data/X_test.csv')

y_train = pd.read_csv('../data/y_train.csv')['y']

# You can check the shapes of the train and test sets

print(f"The train test contains {train.shape[0]} molecules.")
print(f"The test test contains {test.shape[0]} molecules.")

# You can check what it looks like
train.head()

The train test contains 4400 molecules.
The test test contains 2934 molecules.


,id,smiles
0,0,CNC(=O)c1ccccc1Sc1ccc2c(C#Cc3cccc(NCCOC)c3)n[n...
1,1,CC(C)(C)c1ccc(Nc2nnc(-c3cnccc3CCc3ccncc3)o2)cc1
2,2,CN1CCN(CCCn2ccc(-c3cnc4c(-c5ccsc5)cnn4c3)cc2=O...
3,3,CCn1c2ccc(NC(=O)Nc3ccc(OC)cc3)cc2c2c3c(c4c(c21...
4,4,Cc1ccc(C(=O)Nc2cccc(C(C)C)c2)cc1N1CCc2ncncc2C1


As you can see, molecules are encoded using the SMILES notation. To be understandable by the rdkit package, we need to convert them to "Mol" type.

In [12]:
# Convert SMILES to mols
train_mols = [AllChem.MolFromSmiles(smile) for smile in train['smiles']]
test_mols = [AllChem.MolFromSmiles(smile) for smile in test['smiles']]

print(type(train_mols[0]))

<class 'rdkit.Chem.rdchem.Mol'>


After converting SMILES to Mol, we can compute fingerprints. For this example, we use the Morgan fingerprints, a popular type of circular fingerprints, but many others exist. You can choose different parameters for ECFPs, such as the radius, the number of bits, taking intoa ccount chirality or not etc... Do not hesitate to check [this link](https://www.rdkit.org/docs/GettingStartedInPython.html#morgan-fingerprints-circular-fingerprints) to learn more about the different options. 

In [13]:
# Convert Mol to fingerprints
train_fps = np.array([AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2048) for mol in train_mols])
test_fps = np.array([AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2048) for mol in test_mols])

print(f"The train test contains {train_fps.shape[0]} molecules encoded with {train_fps.shape[1]} bits.")
print(f"The test test contains {test_fps.shape[0]} molecules encoded with {test_fps.shape[1]} bits.")

The train test contains 4400 molecules encoded with 2048 bits.
The test test contains 2934 molecules encoded with 2048 bits.


Now that we have the data, it is time to train machine learning models on it. We will be using scikit-learn implementation of Random Forest as predictive model. The ECFPs

In [14]:
X_train = train_fps
X_test = test_fps

### Machine learning

Cross-validation is a good practice to estimate the "true" performance of a model on unseen data. If the cross-validation scores are satisfying, we can then train the model on the entire dataset and predict the test set. For this introduction we use the default parameters of the Random Forest, but keep in mind they might need to be optimized for the challenge. You are encouraged to try other models.

This run can take a few minutes depending on your computer. 

In [15]:
model = RandomForestRegressor(n_jobs=-1)
cv = cross_validate(estimator=model, 
                    X=X_train, 
                    y=y_train, 
                    cv=5, 
                    scoring=["neg_median_absolute_error", "neg_root_mean_squared_error"],
                    n_jobs=-1,
                    verbose=0)

print(f"Mean MAE over 10 folds = {-cv['test_neg_median_absolute_error'].mean():.4f}")
print(f"Mean RMSE over 10 folds = {-cv['test_neg_root_mean_squared_error'].mean():.4f}")

Mean MAE over 10 folds = 0.4288
Mean RMSE over 10 folds = 0.7904


We can now predict X_test and make a submission file (csv) that will later be uploaded on the challenge's website. 

In [16]:
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [17]:
submission_df = pd.DataFrame()
submission_df['id'] = test['id']
submission_df['y'] = y_pred

submission_df.to_csv('../data/y_benchmark.csv', index=False)